# PS Scene and Landsat 8 Crossovers

Have you ever wanted to compare PSScene images to Landsat 8 images? Both image collections are made available via the Planet API. However, it takes a bit of work to identify crossovers - that is, images of the same area that were collected within a reasonable time difference of each other. Also, you may be interested in filtering out some imagery, e.g. cloudy images.

This notebook walks you through the process of finding crossovers between PSScene images and Landsat 8 scenes. In this notebook, we specify 'crossovers' as images that have been taken within 1hr of eachother. This time gap is sufficiently small that we expect the atmospheric conditions won't change much (this assumption doesn't always hold, but is the best we can do for now). We also filter out cloudy images and constrain our search to images collected in 2017, January 1 through August 23.

In [1]:
# Notebook dependencies
from __future__ import print_function

import datetime
import json
import os

import ipyleaflet as ipyl
import ipywidgets as ipyw
from IPython.core.display import HTML
from IPython.display import display
import pandas as pd
from planet import Auth
from planet import Session, DataClient, OrdersClient
from shapely import geometry as sgeom

In [2]:
# if your Planet API Key is not set as an environment variable, you can paste it below
if 'PL_API_KEY' in os.environ:
    API_KEY = os.environ['PL_API_KEY']
else:
    API_KEY = 'PASTE_API_KEY_HERE'
    os.environ['PL_API_KEY'] = API_KEY

client = Auth.from_key(API_KEY)

## Define AOI

Define the AOI as a geojson polygon. This can be done at [geojson.io](http://geojson.io). If you use geojson.io, only copy the single aoi feature, not the entire feature collection.

In [3]:
aoi = {u'type': u'Polygon', u'coordinates': [[[-121.3113248348236, 38.28911976564886], [-121.3113248348236, 38.34622533958], [-121.2344205379486, 38.34622533958], [-121.2344205379486, 38.28911976564886], [-121.3113248348236, 38.28911976564886]]]}

In [4]:
json.dumps(aoi)

'{"type": "Polygon", "coordinates": [[[-121.3113248348236, 38.28911976564886], [-121.3113248348236, 38.34622533958], [-121.2344205379486, 38.34622533958], [-121.2344205379486, 38.28911976564886], [-121.3113248348236, 38.28911976564886]]]}'

## Build Request

Build the Planet API Filter request for the Landsat 8 and PSScene imagery taken in 2017 through August 23.

In [5]:
# Define the filters we'll use to find our data

item_type_LS = ['Landsat8L1G']

item_type_PS = ['PSScene']

geom_filter = {
   "type":"GeometryFilter",
   "field_name":"geometry",
   "config":aoi
}

sun_elevation_filter = {
   "type":"RangeFilter",
   "field_name":"sun_elevation",
   "config":{
       "lt": 0}
}

cloud_cover_filter = {
"type":"RangeFilter",
"field_name":"cloud_cover",
"config":{
  "lt":5 }
}

date_range_filter = {
"type":"DateRangeFilter",
"field_name":"acquired",
"config":{
  "gt":"2017-01-01T00:00:00Z", 
   "lt": "2017-08-23T00:00:00Z"}
}

combined_filter_LS = {
"type":"AndFilter",
"config":[
    geom_filter,
    date_range_filter,
    cloud_cover_filter,
    sun_elevation_filter]
}

combined_filter_PS = {
"type":"AndFilter",
"config":[
    geom_filter,
    date_range_filter,
    cloud_cover_filter]
}


## Search Planet API

The client is how we interact with the planet api. It is created with the user-specific api key, which is pulled from $PL_API_KEY environment variable. Create the client then use it to search for PSScene and Landsat 8 scenes. Save a subset of the metadata provided by Planet API as our 'scene'.

In [6]:
# Run a quick search for our LANDSAT data
async with Session() as sess:
    cl = DataClient(sess)
    results = cl.search(name='landsat_search',search_filter=combined_filter_LS, item_types=item_type_LS)
    landsat_list = [i async for i in results]
    
print(len(landsat_list))
    
# Run a quick search for our PSScene data
async with Session() as sess:
    cl = DataClient(sess)
    results = cl.search(name='ps_search',search_filter=combined_filter_PS, item_types=item_type_PS)
    ps_list = [i async for i in results]
    
print(len(ps_list))

10
100


In processing the items to scenes, we are only using a small subset of the [product metadata](https://www.planet.com/docs/spec-sheets/sat-imagery/#product-metadata). 

In [7]:
def items_to_scenes(items):
    item_types = []

    def _get_props(item):
        props = item['properties']
        props.update({
            'thumbnail': item['_links']['thumbnail'],
            'item_type': item['properties']['item_type'],
            'id': item['id'],
            'acquired': item['properties']['acquired'],
            'footprint': item['geometry']
        })
        return props
    
    scenes = pd.DataFrame(data=[_get_props(i) for i in items])
    
    # acquired column to index, it is unique and will be used a lot for processing
    scenes.index = pd.to_datetime(scenes['acquired'])
    del scenes['acquired']
    scenes.sort_index(inplace=True)
    
    return scenes

## Investigate Landsat Scenes

There are quite a few Landsat 8 scenes that are returned by our query. What do the footprints look like relative to our AOI and what is the collection time of the scenes?

In [8]:
landsat_scenes = items_to_scenes(landsat_list)

# How many Landsat 8 scenes match the query?
print(len(landsat_scenes))

10


### Show Landsat 8 Footprints on Map

In [10]:
def landsat_scenes_to_features_layer(scenes):
    features_style = {
            'color': 'grey',
            'weight': 1,
            'fillColor': 'grey',
            'fillOpacity': 0.15}

    features = [{"geometry": r.footprint,
                 "type": "Feature",
                 "properties": {"style": features_style,
                                "wrs_path": r.wrs_path,
                                "wrs_row": r.wrs_row}}
                for r in scenes.itertuples()]
    return features

def create_landsat_hover_handler(scenes, label):
    def hover_handler(event=None, id=None, properties=None):
        wrs_path = properties['wrs_path']
        wrs_row = properties['wrs_row']
        path_row_query = 'wrs_path=={} and wrs_row=={}'.format(wrs_path, wrs_row)
        count = len(scenes.query(path_row_query))
        label.value = 'path: {}, row: {}, count: {}'.format(wrs_path, wrs_row, count)
    return hover_handler


def create_landsat_feature_layer(scenes, label):
    
    features = landsat_scenes_to_features_layer(scenes)
    
    # Footprint feature layer
    feature_collection = {
        "type": "FeatureCollection",
        "features": features
    }

    feature_layer = ipyl.GeoJSON(data=feature_collection)

    feature_layer.on_hover(create_landsat_hover_handler(scenes, label))
    return feature_layer

In [11]:
# Initialize map using parameters from above map
# and deleting map instance if it exists
try:
    del fp_map
except NameError:
    pass


zoom = 6
center = [38.28993659801203, -120.14648437499999] # lat/lon

In [12]:
# Create map, adding box drawing controls
# Reuse parameters if map already exists
try:
    center = fp_map.center
    zoom = fp_map.zoom
    print(zoom)
    print(center)
except NameError:
    pass

# Change tile layer to one that makes it easier to see crop features
# Layer selected using https://leaflet-extras.github.io/leaflet-providers/preview/
map_tiles = ipyl.TileLayer(url='http://{s}.basemaps.cartocdn.com/light_all/{z}/{x}/{y}.png')
fp_map = ipyl.Map(
        center=center, 
        zoom=zoom,
        default_tiles = map_tiles
    )

label = ipyw.Label(layout=ipyw.Layout(width='100%'))
fp_map.add_layer(create_landsat_feature_layer(landsat_scenes, label)) # landsat layer
fp_map.add_layer(ipyl.GeoJSON(data=aoi)) # aoi layer
    
# Display map and label
ipyw.VBox([fp_map, label])

TypeError: hover_handler() got an unexpected keyword argument 'feature'

TypeError: hover_handler() got an unexpected keyword argument 'feature'

TypeError: hover_handler() got an unexpected keyword argument 'feature'

This AOI is located in a region covered by 3 different path/row tiles. This means there is 3x the coverage than in regions only covered by one path/row tile. This is particularly lucky!

What about the within each path/row tile. How long and how consistent is the Landsat 8 collect period for each path/row?

In [13]:
def time_diff_stats(group):
    time_diff = group.index.to_series().diff() # time difference between rows in group
    stats = {'median': time_diff.median(),
             'mean': time_diff.mean(),
             'std': time_diff.std(),
             'count': time_diff.count(),
             'min': time_diff.min(),
             'max': time_diff.max()}
    return pd.Series(stats)

landsat_scenes.groupby(['wrs_path', 'wrs_row']).apply(time_diff_stats)

median                       mean  \
wrs_path wrs_row                                                         
141      210        15 days 23:59:52.399054 15 days 23:59:52.452082500   
         211     15 days 23:59:52.399054500 15 days 23:59:52.452082666   

                                       std  count                     min  \
wrs_path wrs_row                                                            
141      210     0 days 00:00:01.454449292      6 15 days 23:59:50.567734   
         211     0 days 00:00:01.453739585      6 15 days 23:59:50.576207   

                                     max  
wrs_path wrs_row                          
141      210     15 days 23:59:54.273422  
         211     15 days 23:59:54.273422

It looks like the collection period is 16 days, which lines up with the [Landsat 8 mission description](https://www.usgs.gov/landsat-missions/landsat-8).

path/row 43/33 is missing one image which causes an unusually long collect period.

What this means is that we don't need to look at every Landsat 8 scene collect time to find crossovers with Planet scenes. We could look at the first scene for each path/row, then look at every 16 day increment. However, we will need to account for dropped Landsat 8 scenes in some way.

What is the time difference between the tiles?

In [14]:
import pytz
def find_closest(date_time, data_frame):
    # inspired by:
    # https://stackoverflow.com/questions/36933725/pandas-time-series-join-by-closest-time
    
    # add timezone to datetime object before subtracting
    if date_time.tzinfo is None:
        date_time = pytz.utc.localize(date_time)
    
    
    time_deltas = (data_frame.index - date_time).to_series().reset_index(drop=True).abs()
    #time_deltas = (data_frame.index - pytz.utc.localize(date_time)).to_series().reset_index(drop=True).abs()
    #time_deltas = (data_frame.index.tz_localize(None) - date_time).to_series().reset_index(drop=True).abs()
    idx_min = time_deltas.idxmin()

    min_delta = time_deltas[idx_min]
    return (idx_min, min_delta)

def closest_time(group):
    '''group: data frame with acquisition time as index'''
    inquiry_date = datetime.datetime(year=2017,month=3,day=7)
    idx, _ = find_closest(inquiry_date, group)
    return group.index.to_series().iloc[idx]


# for accurate results, we look at the closest time for each path/row tile to a given time
# using just the first entry could result in a longer time gap between collects due to
# the timing of the first entries

landsat_scenes.groupby(['wrs_path', 'wrs_row']).apply(closest_time)

wrs_path  wrs_row
141       210       2017-03-02 05:55:38.435617+00:00
          211       2017-03-02 05:56:02.318182+00:00
dtype: datetime64[ns, UTC]

So the tiles that are in the same path are very close (36sec) together from the same day. Therefore, we would want to only use one tile and pick the best image.

Tiles that are in different paths are 7 days apart. Therefore, we want to keep tiles from different paths, as they represent unique crossovers.

## Investigate PSScene Images

There are also quite a few PSScene images that match our query. Some of those scenes may not have much overlap with our AOI. We will want to filter those out. Also, we are interested in knowing how many unique days of coverage we have, so we will group PSScene images by collect day, since we may have days with more than one collect (due multiple PS satellites collecting imagery).

In [15]:
all_ps_scenes = items_to_scenes(ps_list)

# How many PS scenes match query?
print(len(all_ps_scenes))
all_ps_scenes[:1]

100


,anomalous_pixels,cloud_cover,ground_control,gsd,instrument,item_type,pixel_resolution,provider,published,publishing_stage,...,satellite_azimuth,satellite_id,strip_id,sun_azimuth,sun_elevation,updated,view_angle,thumbnail,id,footprint
acquired,,,,,,,,,,,,,,,,,,,,,
2017-07-11 18:05:04.155145+00:00,0.01,0.0,True,3.9,PS2,PSScene,3,planetscope,2021-02-13T03:27:25Z,finalized,...,107.7,1004,616044,112.3,58.7,2021-02-13T03:27:25Z,1.2,https://tiles.planet.com/data/v1/item-types/PS...,20170711_180504_1004,"{'coordinates': [[[-121.35906151531412, 38.436..."


What about overlap? We really only want images that overlap over 20% of the AOI.

Note: we do this calculation in WGS84, the geographic coordinate system supported by geojson. The calculation of coverage expects that the geometries entered are 2D, which WGS84 is not. This will cause a small inaccuracy in the coverage area calculation, but not enough to bother us here.

In [16]:
def aoi_overlap_percent(footprint, aoi):
    aoi_shape = sgeom.shape(aoi)
    footprint_shape = sgeom.shape(footprint)
    overlap = aoi_shape.intersection(footprint_shape)
    return overlap.area / aoi_shape.area

overlap_percent = all_ps_scenes.footprint.apply(aoi_overlap_percent, args=(aoi,))
all_ps_scenes = all_ps_scenes.assign(overlap_percent = overlap_percent)
all_ps_scenes.head()

,anomalous_pixels,cloud_cover,ground_control,gsd,instrument,item_type,pixel_resolution,provider,published,publishing_stage,...,satellite_id,strip_id,sun_azimuth,sun_elevation,updated,view_angle,thumbnail,id,footprint,overlap_percent
acquired,,,,,,,,,,,,,,,,,,,,,
2017-07-11 18:05:04.155145+00:00,0.01,0.0,True,3.9,PS2,PSScene,3,planetscope,2021-02-13T03:27:25Z,finalized,...,1004,616044,112.3,58.7,2021-02-13T03:27:25Z,1.2,https://tiles.planet.com/data/v1/item-types/PS...,20170711_180504_1004,"{'coordinates': [[[-121.35906151531412, 38.436...",0.045713
2017-07-11 18:05:06.259145+00:00,0.01,0.0,True,3.9,PS2,PSScene,3,planetscope,2021-02-13T03:27:23Z,finalized,...,1004,616044,112.1,58.7,2021-02-13T03:27:23Z,1.2,https://tiles.planet.com/data/v1/item-types/PS...,20170711_180506_1004,"{'coordinates': [[[-121.39749625848998, 38.304...",0.000031
2017-07-12 18:06:29.181869+00:00,0.01,0.0,True,3.9,PS2,PSScene,3,planetscope,2021-02-13T03:55:05Z,finalized,...,1033,619625,112.6,58.5,2021-02-13T03:55:05Z,2.6,https://tiles.planet.com/data/v1/item-types/PS...,20170712_180629_1033,"{'coordinates': [[[-121.37369937311475, 38.419...",0.372850
2017-07-12 18:06:30.225784+00:00,0.01,0.0,True,3.9,PS2,PSScene,3,planetscope,2021-02-13T03:55:08Z,finalized,...,1033,619625,112.5,58.6,2021-02-13T03:55:08Z,2.5,https://tiles.planet.com/data/v1/item-types/PS...,20170712_180630_1033,"{'coordinates': [[[-121.39260264240319, 38.354...",0.772277
2017-07-13 18:06:17.048410+00:00,0.01,0.0,True,3.9,PS2,PSScene,3,planetscope,2021-02-13T04:53:03Z,finalized,...,1003,621263,112.7,58.4,2021-02-13T04:53:03Z,3.2,https://tiles.planet.com/data/v1/item-types/PS...,20170713_180617_1003,"{'coordinates': [[[-121.27662044116802, 38.387...",0.509247


In [17]:
print(len(all_ps_scenes))
ps_scenes = all_ps_scenes[all_ps_scenes.overlap_percent > 0.20]
print(len(ps_scenes))

100
68


Ideally, PS scenes have daily coverage over all regions. How many days have PS coverage and how many PS scenes were taken on the same day?

In [18]:
# ps_scenes.index.to_series().head()
# ps_scenes.filter(items=['id']).groupby(pd.Grouper(freq='D')).agg('count')

In [19]:
# Use PS acquisition year, month, and day as index and group by those indices
# https://stackoverflow.com/questions/14646336/pandas-grouping-intra-day-timeseries-by-date
daily_ps_scenes = ps_scenes.index.to_series().groupby([ps_scenes.index.year,
                                                       ps_scenes.index.month,
                                                       ps_scenes.index.day])

In [20]:
daily_count = daily_ps_scenes.agg('count')
daily_count.index.names = ['y', 'm', 'd']

# How many days is the count greater than 1?
daily_multiple_count = daily_count[daily_count > 1]

print('Out of {} days of coverage, {} days have multiple collects.'.format( \
    len(daily_count), len(daily_multiple_count)))

daily_multiple_count.head()

Out of 26 days of coverage, 21 days have multiple collects.


y     m  d 
2017  7  12    2
         13    6
         16    2
         17    3
         18    2
Name: acquired, dtype: int64

In [21]:
def scenes_and_count(group):
    entry = {'count': len(group),
             'acquisition_time': group.index.tolist()}
    return pd.DataFrame(entry)

daily_count_and_scenes = daily_ps_scenes.apply(scenes_and_count)
# need to rename indices because right now multiple are called 'acquired', which
# causes a bug when we try to run the query
daily_count_and_scenes.index.names = ['y', 'm', 'd', 'num']

multiplecoverage = daily_count_and_scenes.query('count > 1')

multiplecoverage.query('m == 7')  # look at just occurrence in July

count                 acquisition_time
y    m d  num                                        
2017 7 12 0        2 2017-07-12 18:06:29.181869+00:00
          1        2 2017-07-12 18:06:30.225784+00:00
       13 0        6 2017-07-13 18:06:17.048410+00:00
          1        6 2017-07-13 18:06:18.100410+00:00
          2        6 2017-07-13 18:07:40.302462+00:00
          3        6 2017-07-13 18:07:41.338462+00:00
          4        6 2017-07-13 18:07:49.840306+00:00
          5        6 2017-07-13 18:07:50.882306+00:00
       16 0        2 2017-07-16 18:06:32.406464+00:00
          1        2 2017-07-16 18:06:33.449464+00:00
       17 0        3 2017-07-17 18:05:41.353902+00:00
          1        3 2017-07-17 18:07:58.214699+00:00
          2        3 2017-07-17 18:07:59.266699+00:00
       18 0        2 2017-07-18 18:12:51.958982+00:00
          1        2 2017-07-18 18:12:53.022982+00:00
       20 0        3 2017-07-20 18:06:20.855891+00:00
          1        3 2017-07-20 18:06:51.308177+00:00
          2        3 2017-07-20 18:06:52.367405+00:00
       21 0        2 2017-07-21 18:06:25.235750+00:00
          1        2 2017-07-21 18:06:26.287145+00:00
       25 0        2 2017-07-25 18:06:03.194477+00:00
          1        2 2017-07-25 18:06:04.230011+00:00
       26 0        4 2017-07-26 18:06:39.940367+00:00
          1        4 2017-07-26 18:06:40.991971+00:00
          2        4 2017-07-26 18:09:11.120062+00:00
          3        4 2017-07-26 18:09:12.176802+00:00
       27 0        4 2017-07-27 18:06:40.586389+00:00
          1        4 2017-07-27 18:06:41.638778+00:00
          2        4 2017-07-27 18:07:32.145195+00:00
          3        4 2017-07-27 18:07:33.205533+00:00
       28 0        4 2017-07-28 18:06:58.513592+00:00
          1        4 2017-07-28 18:06:59.551114+00:00
          2        4 2017-07-28 18:07:50.843765+00:00
          3        4 2017-07-28 18:07:51.902431+00:00
       29 0        2 2017-07-29 18:06:42.712533+00:00
          1        2 2017-07-29 18:06:43.755733+00:00
       31 0        2 2017-07-31 18:05:16.666047+00:00
          1        2 2017-07-31 18:05:17.718314+00:00

Looks like the multiple collects on the same day are just a few minutes apart. They are likely crossovers between different PS satellites. Cool! Since we only want to us one PS image for a crossover, we will chose the best collect for days with multiple collects.

## Find Crossovers

Now that we have the PSScene filtered to what we want and have investigated the Landsat 8 scenes, let's look for crossovers between the two.

First we find concurrent crossovers, PS and Landsat collects that occur within 48hours of each other.

In [30]:
def find_crossovers(acquired_time, landsat_scenes):
    '''landsat_scenes: pandas dataframe with acquisition time as index'''
    closest_idx, closest_delta = find_closest(acquired_time, landsat_scenes)
    closest_landsat = landsat_scenes.iloc[closest_idx]

    crossover = {'landsat_acquisition': closest_landsat.name,
                 'delta': closest_delta}
    return pd.Series(crossover)


# fetch PS scenes
ps_scenes = items_to_scenes(ps_list)


# for each PS scene, find the closest Landsat scene
# Remove timezone info from the datetime index in order to subtract
crossovers = ps_scenes.index.to_series().apply(find_crossovers, args=(landsat_scenes,))

# filter to crossovers within 90 days
concurrent_crossovers = crossovers[crossovers['delta'] < pd.Timedelta('90 days')]
print(len(concurrent_crossovers))
concurrent_crossovers

20


,landsat_acquisition,delta
acquired,,
2017-07-11 18:05:04.155145+00:00,2017-04-19 05:55:36.338299+00:00,83 days 12:09:27.816846
2017-07-11 18:05:06.259145+00:00,2017-04-19 05:55:36.338299+00:00,83 days 12:09:29.920846
2017-07-12 18:06:29.181869+00:00,2017-04-19 05:55:36.338299+00:00,84 days 12:10:52.843570
2017-07-12 18:06:30.225784+00:00,2017-04-19 05:55:36.338299+00:00,84 days 12:10:53.887485
2017-07-13 18:06:17.048410+00:00,2017-04-19 05:55:36.338299+00:00,85 days 12:10:40.710111
2017-07-13 18:06:18.100410+00:00,2017-04-19 05:55:36.338299+00:00,85 days 12:10:41.762111
2017-07-13 18:07:40.302462+00:00,2017-04-19 05:55:36.338299+00:00,85 days 12:12:03.964163
2017-07-13 18:07:41.338462+00:00,2017-04-19 05:55:36.338299+00:00,85 days 12:12:05.000163
2017-07-13 18:07:49.840306+00:00,2017-04-19 05:55:36.338299+00:00,85 days 12:12:13.502007


Now that we have the crossovers, what we are really interested in is the IDs of the landsat and PS scenes, as well as how much they overlap the AOI.

In [31]:
def get_crossover_info(crossovers, aoi):
    def get_scene_info(acquisition_time, scenes):
        scene = scenes.loc[acquisition_time]
        scene_info = {'id': scene.id,
                      'thumbnail': scene.thumbnail,
                      # we are going to use the footprints as shapes so convert to shapes now
                      'footprint': sgeom.shape(scene.footprint)}
        return pd.Series(scene_info)

    landsat_info = crossovers.landsat_acquisition.apply(get_scene_info, args=(landsat_scenes,))
    ps_info = crossovers.index.to_series().apply(get_scene_info, args=(ps_scenes,))

    footprint_info = pd.DataFrame({'landsat': landsat_info.footprint,
                                   'ps': ps_info.footprint})
    overlaps = footprint_info.apply(lambda x: x.landsat.intersection(x.ps),
                                    axis=1)
    
    aoi_shape = sgeom.shape(aoi)
    overlap_percent = overlaps.apply(lambda x: x.intersection(aoi_shape).area / aoi_shape.area)
    crossover_info = pd.DataFrame({'overlap': overlaps,
                                   'overlap_percent': overlap_percent,
                                   'ps_id': ps_info.id,
                                   'ps_thumbnail': ps_info.thumbnail,
                                   'landsat_id': landsat_info.id,
                                   'landsat_thumbnail': landsat_info.thumbnail})
    return crossover_info

crossover_info = get_crossover_info(concurrent_crossovers, aoi)
print(len(crossover_info))

20


Next, we check to see if there are overlaps that cover a significant portion of the AOI. In this case there are not.

In [36]:
significant_crossovers_info = crossover_info[crossover_info.overlap_percent > 0.9]
print(len(significant_crossovers_info))

0


Browsing through the crossovers, we see that in some instances, multiple crossovers take place on the same day. Really, we are interested in 'unique crossovers', that is, crossovers that take place on unique days. Therefore, we will look at the concurrent crossovers by day.

In [37]:
def group_by_day(data_frame):
    return data_frame.groupby([data_frame.index.year,
                               data_frame.index.month,
                               data_frame.index.day])

unique_crossover_days = group_by_day(crossover_info.index.to_series()).count()
print(len(unique_crossover_days))
print(unique_crossover_days)

7
acquired  acquired  acquired
2017      7         11          2
                    12          2
                    13          6
                    14          2
                    15          2
                    16          2
                    17          4
Name: acquired, dtype: int64


There are 7 unique crossovers between Landsat 8 and PS that cover over 90% of our AOI between January and August in 2017. Not bad! That is definitely enough to perform comparison.

### Display Crossovers

Let's take a quick look at the crossovers we found to make sure that they don't look cloudy, hazy, or have any other quality issues that would affect the comparison.

In [38]:
# https://stackoverflow.com/questions/36006136/how-to-display-images-in-a-row-with-ipython-display
def make_html(image):
     return '<img src="{0}" alt="{0}"style="display:inline;margin:1px"/>' \
            .format(image)


def display_thumbnails(row):
    print(row.name)
    display(HTML(''.join(make_html(t)
                         for t in (row.ps_thumbnail, row.landsat_thumbnail))))

_ = crossover_info.apply(display_thumbnails, axis=1)

2017-07-11 18:05:04.155145+00:00


2017-07-11 18:05:06.259145+00:00


2017-07-12 18:06:29.181869+00:00


2017-07-12 18:06:30.225784+00:00


2017-07-13 18:06:17.048410+00:00


...

2017-07-16 18:06:33.449464+00:00


2017-07-17 18:05:40.294902+00:00


2017-07-17 18:05:41.353902+00:00


2017-07-17 18:07:58.214699+00:00


2017-07-17 18:07:59.266699+00:00


They all look pretty good although the last crossover (2017-08-10) could be a little hazy.